In [ ]:
!pip install geetools 
!pip install geemap
!pip install rasterio
!pip install retry

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 74 kB 2.3 MB/s 
     |████████████████████████████████| 46 kB 3.8 MB/s 
  Created wheel for geetools: filename=geetools-0.6.14-py3-none-any.whl size=92107 sha256=0be27dffe2642c15b0e55c2e3f73a43c269b2f2388f7a72744055b18c9a8a532
  Stored in directory: /root/.cache/pip/wheels/5c/55/29/0a09dcf6b39bba8c890e40171cac3e3607a5c4b354b7e9447c
Successfully built geetools
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.2 MB/s 
     |████████████████████████████████| 187 kB 41.6 MB/s 
     |████████████████████████████████| 130 kB 49.5 MB/s 
     |████████████████████████████████| 3.3 MB 38.0 MB/s 
     |████████████████████████████████| 3.0 MB 37.8 MB/s 
     |████████████████████████████████| 1.3 MB 16.6 MB/s 
     |████████████████████████████████| 1.2 MB 43.2 M

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 1.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import ee
import csv
import pandas as pd
import numpy as np
import time
import geemap
from geetools import batch
from tqdm import tqdm
from argparse import ArgumentParser
import requests
import matplotlib.pyplot as plt
import folium
import math
import rasterio
import logging
import multiprocessing
import requests
import shutil
from retry import retry
import multiprocessing

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=5-cUQLAbE7vb_Ca2SUPnHbM-suHpyY7LAxOtR1-tTNA&tc=OVrxLZt3UQ_Iqx-Bm7SY_Jq-v62skFRWcD6LbA24u5A&cc=6Dn-ejtYWVKbE4rdkKKJrscwGfNvKs8rg-pf76E5jo4

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWg-PjBH3BqTcKKZFJcvthvh6pN8fjNZcVTAiQSsD0UsEGh1cQLkp9w

Successfully saved authorization token.


In [ ]:
# Initialize the library.
##ee.Initialize()
##high volume API 
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


In [ ]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3)

# Add the elevation model to the map object.
my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [ ]:
##might have to come up with different formula 
## concern 1: does this result in the desired pixel dimesions of image
## concern 2: should we not use radial distance instead? 
# def getSquareImageCorners(lat, lon, height, width):
#     offset = 0.2
#     xMin = lon - offset
#     xMax = lon + offset  
#     yMin = lat - offset
#     yMax = lat + offset
#     return xMin, xMax, yMin, yMax 

In [ ]:
## find a bounding box
# try to calculate the dimension of the 512 * 512 image using dimension
# images don't result in desired size
# resolution = input()
resLandsat = 30 
def boundingBox(lat, lon, size, res):

  earth_radius = 6371000
  angular_distance = math.degrees(0.5 * ((size * res) / earth_radius))
  osLat = angular_distance
  osLon = angular_distance ##/ math.cos(math.radians(lat))
  xMin = lon - osLon
  xMax = lon + osLon
  yMin = lat - osLat
  yMax = lat + osLat
  return xMin, xMax, yMin, yMax

In [ ]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")

In [ ]:
start_date = '2017-01-01'
end_date = '2018-01-01'

In [ ]:
RGB_VIZ = {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':0.4}

res = 30 

crs = 'EPSG:3857'
crs2 = 'EPSG:4326'
fileFormat = 'GeoTIFF'
cloud_filter = 10

#   with open(filepath) as coords_file:
#     coords = csv.reader(coords_file)
#     for coord in coords:
#         lat = coord[0]
#         lon = coord[1]


lat = 32.568364733544115
lon = -11.290082825465532
# lat = 35.994034
# lon = -78.898621
description = 'test_image_0'
start='2017-01-01'
end='2020-01-01'
height = 512

xMin, xMax, yMin, yMax = boundingBox(lat, lon, height, res)
geometry = ee.Geometry.Rectangle([[xMin, yMin],[xMax, yMax]])

filtered = landsat.filterDate(start, end).filterBounds(geometry).select(['B2', 'B3', 'B4', 'B5'])
try:
  image = filtered.median().clip(geometry)
  map = folium.Map(location=[lat, lon], zoom_start=10)
  map.add_ee_layer(image, RGB_VIZ, 'rgb')
  display(map)
except ee.EEException:
  print("image missing one or more bands")
  #print(image.getInfo())
  image = filtered.median().clip(geometry)
  map.add_ee_layer(image, {}, 'rgb')
  display(map)
  print(image.getInfo())
#image = image.visualize(RGB_VIZ)




In [88]:
num = 100000
coords_file = open(f'generated_coords_{num}.csv', 'w', newline='')
writer = csv.writer(coords_file)
header = ['lat', 'lon']
writer.writerow(header)
start = time.time()
for i in range(num):
  lat = np.random.default_rng().uniform(low=-90.0, high=90.0, size=None)
  lon = np.random.default_rng().uniform(low=-180.0, high=180.0, size=None)
  writer.writerow([lat, lon])
end = time.time()
coords_file.close()
print(f'to generate {num} coordinates, it took: {end-start}s')


to generate 100000 coordinates, it took: 10.375426530838013s


In [78]:
# Attempt to use logging
logger = logging.getLogger('my_logger')
logging.basicConfig(filename = 'file.log',
                    filemode = 'a' #append mode (maybe should be w instead?)
                    # ,format = ?
                    )
logger.debug('This is my debug message ')
logger.info('This is my info message ')
logger.warning('This is my warning message ')
logger.error('This is my error message ')
logger.critical('This is my critical message ')


In [79]:
# to use later for parallel processing

# @retry(tries=10, delay=1, backoff=2)
# def export(url, description):
# # save image to disk!
#   response = requests.get(url)
#   with open(f'{description}_thumbnail.tif', 'wb') as fd:
#     fd.write(response.content)


In [80]:
def extract_images(filepath=f'/content/generated_coords_{num}.csv',
                    listOfCoords=[(35.994034, -78.898621)], 
                    dataset=landsat, 
                    start='2017-01-01', end='2020-01-01', 
                    bands=['B2', 'B3', 'B4', 'B5'], 
                    height=512, 
                    width=512):

  ''' TODO:

          add default values
          add number of images to get param + interval between images? --> multi-temporal views (default=1)
          change listOfCoords to file that has all coords
          add support for pan-sharpening
  '''

  RGB_VIZ = {'min':0, 'max':0.4, 'bands':['B4', 'B3', 'B2']}
  res = 30 if dataset == landsat else 10
  crs = 'EPSG:3857'
  crs2 = 'EPSG:4326'
  fileFormat = 'GeoTIFF'
  cloud_percent = 20
  # Define the visualization parameters.
  image_viz_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.5}

  coords = pd.read_csv(filepath)
  total_duration = 0
  start_time_recorded = False

  for i in tqdm(range(len(coords))):
    coord = ((coords["lat"][i], coords["lon"][i]))
    lat = coord[0]
    lon = coord[1]

    description = f'image_{lat}_{lon}'
    xMin, xMax, yMin, yMax = boundingBox(lat, lon, height, res)
    geometry = ee.Geometry.Rectangle([[xMin, yMin],[xMax, yMax]])
    RGB = ['B4', 'B3', 'B2'] if dataset == landsat else ['R', 'G', 'B']

    filtered = dataset.filterDate(start, end).filterBounds(geometry).filter(ee.Filter.lt('CLOUD_COVER', cloud_filter)).select(bands)
    try:
      image = filtered.median().clip(geometry).select(RGB)
       ## LANDSAT SPECIFIC!! MUST CHANGE MIN, MAX PARAMS DEPENDING ON MEANING IN EACH DATASET 

      img = image.visualize(bands = RGB, min = 0, max = 0.5)
      
      
      # OPTIONAL: just to view the map, to comment out when exporting more than 1 image
      # map = folium.Map(location=[lat, lon], zoom_start=10)
      # map.add_ee_layer(image, image_viz_params, 'rgb')
      # display(map)

      export_links = True

  # run this if we want to export links
      if export_links == True:
        url = image.getDownloadUrl({
        'description':description,
        'region':geometry,
        'fileNamePrefix':description,
        'crs':crs,
        'fileFormat':fileFormat,
        'bands': ['B4', 'B3', 'B2'],
        'region': geometry,
        'format': 'GEO_TIFF',
        #'scale': res,
        'dimensions': [height, width]
        })

      ## when exported using scale and region, images are not always 512x512, whereas specifiying dimesions forces the image to be of requested dims
      ## exporting images using getDownloadUrl reuslts in ~700 kb files (thumbnails), whereas exporting to Drive results in 1 Mb images

    # save image to disk!
      if not start_time_recorded:
        start_time_recorded = True
        export_start_time = time.time()
      response = requests.get(url)
      with open(f'{description}.tif', 'wb') as fd:
        fd.write(response.content)

      # # save thumbnail to disk
      # local_save = geemap.ee_export_image(img, 
      #                       filename=description+'_thumbnail.tif', 
      #                       scale=res, 
      #                       region=geometry,
      #                       file_per_band=False)
      

        
      # #save to Drive of logged-in Google account 
      # task = ee.batch.Export.image.toDrive(image=img,
      #                                  description=description,
      #                                  #scale=res, 
      #                                  region=geometry,
      #                                  fileNamePrefix=description,
      #                                  crs=crs,
      #                                  fileFormat=fileFormat,
      #                                  folder='June 8 test 2.0',
      #                                 ## CANNOT SPECIFY BOTH SCALE AND DIMENSIONS
      #                                 dimensions = height)
      
      task = None
      ## LOOK INTO BUFFER

      # try logging for each image downloaded
      # logger
      
      ##task.start()
      ## Note: this time would be more accurate for Drive saving
      ## if the images are not saved locally too, as that is also added to the total duration 
      ##print('Exporting to Google Drive started!')

    except ee.EEException:
      print(f"\nimage at {(lat, lon)} missing one or more RGB bands")
      pass

  return task, export_start_time


In [89]:
coordinates = f'/content/generated_coords_{num}.csv.csv'

In [ ]:
'''
TODO :
      * write function to generate items (list of coords)
      * write function that takes items as param (and requests images)
      * experiment with different numbers of parallel processes (25 currently)

This will be used to make requests using parallel processes and speed up the requests
Items should be the list of coordinates


'''

# items = None
# pool = multiprocessing.Pool(25)
# pool.starmap(extract_images, enumerate(items))
# pool.close()
# pool.join()

In [83]:
mytask, export_start_time = extract_images(filepath=coordinates,
                    listOfCoords=[], 
                    dataset=landsat, 
                    start=start_date, 
                    end=end_date, 
                    bands=['B2', 'B3', 'B4', 'B5'], 
                    height=512, 
                    width=512)
export_finish_time = time.time()
duration = export_finish_time - export_start_time
print(f'exporting took: {duration} s')

  2%|▏         | 2/100 [00:02<01:47,  1.10s/it]


image at (-77.68399841425418, 131.71642984431912) missing one or more RGB bands


  6%|▌         | 6/100 [00:12<03:01,  1.94s/it]


image at (-86.81765772154516, 172.39755218380122) missing one or more RGB bands


  8%|▊         | 8/100 [00:13<01:31,  1.00it/s]

...


image at (-81.61294739604071, 87.21864541529635) missing one or more RGB bands


 98%|█████████▊| 98/100 [02:42<00:01,  1.69it/s]


image at (-69.71264942651261, 149.44966818347234) missing one or more RGB bands


100%|██████████| 100/100 [02:43<00:00,  1.63s/it]


image at (-25.30411077940215, -170.0847350532431) missing one or more RGB bands

image at (87.54414219616157, 133.31281269404354) missing one or more RGB bands
exporting took: 162.83494567871094 s


In [ ]:
# ## to calculate export times to drive
# state = mytask.status()['state']
# previous = state
# print(state)
# while state != 'COMPLETED':
#   state = mytask.status()['state']
#   if state != previous:
#     print(state)
#     previous = state
#   if state == 'FAILED':
#     break
# print('export complete!')
# export_finish_time = time.time()
# duration = export_finish_time - export_start_time
# print(f'exporting took: {duration} s')

COMPLETED
export complete!
exporting took: 463.001859664917 s


In [ ]:
## view metadata of export
%%bash
gdalinfo /content/image_14.tif

Driver: GTiff/GeoTIFF
Files: /content/image_14.tif
Size is 614, 514
Coordinate System is:
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]
Origin = (44.283350245955937,33.384450229847431)
Pixel Size = (0.000269494585236,-0.000269494585236)
Metadata:
  AREA_OR_POINT=Area
  TIFFTAG_RESOLUTIONUNIT=1 (unitless)
  TIFFTAG_XRESOLUTION=1
  TIFFTAG_YRESOLUTION=1
Image Structure Metadata:
  COMPRESSION=DEFLATE
  INTERLEAVE=PIXEL
Corner Coordinates:
Upper Left  (  44.2833502,  33.3844502) ( 44d17' 0.06"E, 33d23' 4.02"N)
Lower Left  (  44.2833502,  33.2459300) ( 44d17' 0.06"E, 33d14'45.35"N)
Upper Right (  44.4488199,  33.3844502) ( 44d26'55.75"E, 33d23' 4.02"N)
Lower Right (  44.4488199,  33.2459300) ( 44d26'55.75"E, 33d14'45.35"N)
Center      (  44.3660851,  33.3151901) ( 44d21'57.91"E, 33